# List comprehension vs for loop + append

In [11]:
import sys
print(sys.version)

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]


In [12]:
import dis
import timeit

# 1. 개요
## python은 **인터프리터 언어**이지만 인터프리터 내부적으로 컴파일을 실행하고 컴파일된 **바이트 코드를 한줄씩 기계어로 변환해 수행**
- 인터프리팅과 컴파일이라는 개념은 언어적인 특성이라기보단 **구현 방식의 문제**
- 언어를 어떻게 구현하고 수정하느냐에 따라 인터프리터와 컴파일러의 동작이 들어갈수도 있고 아닐 수도 있다
  - 프로그래밍 언어는 런타임(컴파일러, 인터프리터 등의 실행 소프트웨어)를 제어하는 룰의 모음이다
    - 여기서 언급한 런타임이란 프로그램이 실행되는 환경
  - 현재 Python(3.11.5 버전)의 공식 런타임/구현체는 CPython이며, CPython에서는 실제 기계어로 한 문장씩 수행하기 전에 바이트코드로 컴파일하여 재사용함
## **dis 모듈**은 인터프리팅되어 실행되는 기계어를, 역어셈블리 변환을 통해 **바이트 코드**로 변환 후 출력해주는 모듈
- 바이트 코드는 플랫폼(운영체제) 독립적인 코드: 동일한 바이트 코드를 윈도우용 VM에서도 수행할 수 있고, 리눅스용 VM에서도 수행할 수 있다. 공유가 가능하다.
## 해당 모듈을 통해 반환되는 바이트 코드와, cpython코드를 분석해 list comprehension 과 for loop + append 동작을 비교해보자.
---

# 2. 수행 시간 비교 

In [13]:
def for_loop():
    list = []
    for i in range(100_000):
        list.append(i)
    return list

In [14]:
def list_comprehension():
    return [i for i in range(100_000)]

In [15]:
for_loop_time = timeit.timeit(lambda: for_loop(), number=1_000)

In [16]:
list_comprehension_time = timeit.timeit(lambda: list_comprehension(), number=1_000)

In [17]:
print(list_comprehension_time / for_loop_time * 100)

66.97440432446449


## 결론: 1억개 엔티티 리스트 생성시 list comprehension이 약 34% 정도 빠르다.
---

# 3. 바이트 코드 비교

In [7]:
dis.dis(for_loop)

  1           0 RESUME                   0

  2           2 BUILD_LIST               0
              4 STORE_FAST               0 (list)

  3           6 LOAD_GLOBAL              1 (NULL + range)
             18 LOAD_CONST               1 (100000)
             20 PRECALL                  1
             24 CALL                     1
             34 GET_ITER
        >>   36 FOR_ITER                23 (to 84)
             38 STORE_FAST               1 (i)

  4          40 LOAD_FAST                0 (list)
             42 LOAD_METHOD              1 (append)
             64 LOAD_FAST                1 (i)
             66 PRECALL                  1
             70 CALL                     1
             80 POP_TOP
             82 JUMP_BACKWARD           24 (to 36)

  5     >>   84 LOAD_FAST                0 (list)
             86 RETURN_VALUE


In [8]:
dis.dis(list_comprehension)

  1           0 RESUME                   0

  2           2 LOAD_CONST               1 (<code object <listcomp> at 0x0000017B0021B5D0, file "C:\Users\jsh\AppData\Local\Temp\ipykernel_5696\4287737444.py", line 2>)
              4 MAKE_FUNCTION            0
              6 LOAD_GLOBAL              1 (NULL + range)
             18 LOAD_CONST               2 (100000)
             20 PRECALL                  1
             24 CALL                     1
             34 GET_ITER
             36 PRECALL                  0
             40 CALL                     0
             50 RETURN_VALUE

Disassembly of <code object <listcomp> at 0x0000017B0021B5D0, file "C:\Users\jsh\AppData\Local\Temp\ipykernel_5696\4287737444.py", line 2>:
  2           0 RESUME                   0
              2 BUILD_LIST               0
              4 LOAD_FAST                0 (.0)
        >>    6 FOR_ITER                 4 (to 16)
              8 STORE_FAST               1 (i)
             10 LOAD_FAST          

- 둘을 비교해보면 차이나는 부분이 두가지
- 2 ~ 4 라인: BUILD_LIST & STORE_FAST vs LOAD_CONST & MAKE_FUNCTION
  - BUILD_LIST: 리스트 생성
  - STORE_FAST(var_num): TOS를 지역 co_varnames[var_num]에 저장합니다.
    - TOS: Top Of Stack
  - LOAD_CONST(consti): co_consts[consti]를 스택으로 푸시합니다.
  - MAKE_FUNCTION(flags): 스택에 새 함수 객체를 푸시합니다. 바닥에서 맨 위로, 인자가 지정된 플래그 값을 전달하면 소비되는 스택은 값으로 구성되어야 합니다.
    - ref: https://docs.python.org/ko/3.11/library/dis.html
  - for loop 에서는 리스트를 생성해 스택에 로드
  - list comprehension에서는 listcomp라는 함수객체를 생성해 스택에 로드
- 36 이후 라인: FOR_ITER vs PRE_CALL & CALL
  - for loop 에서는 range()의 호출 후 range iterator를 생성해 next를 호출하며, 38 ~ 82 라인을 반복 수행
    - TOS인 iterable객체(range(100_000)) 를 수행한 결과를 인자로 append 메서드를 호출해 수행함
  - list comprehension에서는 range() 의 결과 iterator를 생성해 스택에 있는 callable 객체들(여기선 listcomp 함수객체)를 차례대로 모두 수행(call)하고 반환값이 있으면 stack 에 push
    - listcomp 함수가 호출되어 아래의 코드가 차례대로 수행
    - list생성 후 TOS인 iterable 객체(range(100_000)) 를 수행한 결과를 차례로 LIST_APPEND 처리함

## 결론: 위 두가지 차이점에서 수행 속도에 영향을 받을 수 있는 부분은 append 메서드 호출과 LIST_APPEND 처리 부분
### 가정 1. append 메서드가 LIST_APPEND보다 느리다.
### 가정 2. 메서드를 LOAD하고 CALL하는 부분이 느리다.
---

# 3-1. append 메서드와 LIST_APPEND 바이트 코드 비교
- LIST_APPEND(i): list.append(TOS1[-i], TOS)를 호출합니다. 리스트 컴프리헨션을 구현하는 데 사용됩니다.
- https://github.com/python/cpython/blob/7b55a955bc4066c5db4c842b3d7319df487fb750/Python/ceval.c#L2268C1-L2269C1
- https://github.com/python/cpython/blob/fa84e5fe0a3bd8e77c33b20867d71ac6bee270f9/Objects/listobject.c#L322
## append
``` C
/*[clinic input]
list.append

     object: object
     /

Append object to the end of the list.
[clinic start generated code]*/

static PyObject *
list_append(PyListObject *self, PyObject *object)
/*[clinic end generated code: output=7c096003a29c0eae input=43a3fe48a7066e91]*/
{
    if (_PyList_AppendTakeRef(self, Py_NewRef(object)) < 0) {
        return NULL;
    }
    Py_RETURN_NONE;
}
```

## LIST_APPEND
``` C
TARGET(LIST_APPEND) {
     PyObject *v = POP();
     PyObject *list = PEEK(oparg);
     if (_PyList_AppendTakeRef((PyListObject *)list, v) < 0)
         goto error;
     PREDICT(JUMP_BACKWARD_QUICK);
     DISPATCH();
}
```
## 결론: LIST_APPEND가 stack 에서 리스트를 꺼내오고 이전 수행 위치로 돌아가는 연산이 있긴 하지만 append 자체는 동일한 연산을 수행한다.(_PyList_AppendTakeRef)
---

# 3-2. 메서드 LOAD + CALL 의 수행 속도 측정
- pass 를 사용해 아무 연산도 하지 않고 메서드 Load 와 call만을 수행하는 함수를 생성

In [26]:
def empty():
    pass

- for 문으로 아무것도 하지 않는 코드와 empty()를 호출하는 코드의 수행 시간을 비교해보자.

In [41]:
def no_function_call():
    for i in range(100_000_000):
        pass

In [42]:
def with_function_call():
    for i in range(100_000_000):
        empty()

In [44]:
no_func_time = timeit.timeit(lambda: no_function_call(), number=1)
with_func_time = timeit.timeit(lambda: with_function_call(), number=1)

In [53]:
print(f"메서드 콜이 포함된 연산 시간이, 메서드 콜이 없는 연산 시간의 {with_func_time / no_func_time * 100:.2f}% 입니다.")

메서드 콜이 포함된 연산 시간이, 메서드 콜이 없는 연산 시간의 354.45% 입니다.


In [50]:
dis.dis(no_function_call)

  1           0 RESUME                   0

  2           2 LOAD_GLOBAL              1 (NULL + range)
             14 LOAD_CONST               1 (100000000)
             16 PRECALL                  1
             20 CALL                     1
             30 GET_ITER
        >>   32 FOR_ITER                 2 (to 38)
             34 STORE_FAST               0 (i)

  3          36 JUMP_BACKWARD            3 (to 32)

  2     >>   38 LOAD_CONST               0 (None)
             40 RETURN_VALUE


In [51]:
dis.dis(with_function_call)

  1           0 RESUME                   0

  2           2 LOAD_GLOBAL              1 (NULL + range)
             14 LOAD_CONST               1 (100000000)
             16 PRECALL                  1
             20 CALL                     1
             30 GET_ITER
        >>   32 FOR_ITER                16 (to 66)
             34 STORE_FAST               0 (i)

  3          36 LOAD_GLOBAL              3 (NULL + empty)
             48 PRECALL                  0
             52 CALL                     0
             62 POP_TOP
             64 JUMP_BACKWARD           17 (to 32)

  2     >>   66 LOAD_CONST               0 (None)
             68 RETURN_VALUE


- 두 함수의 바이트 코드를 비교해보면 함수를 로드하고 호출하는 부분만 추가되었음을 확인할 수 있다

## 결론: for loop + append 가 list comprehension에 비해 느린 이유는, for loop + append에서 append 호출시 발생하는 메서드 콜 연산 오버헤드가 크기 때문이다. (== list comprehension 의  메서드 콜 횟수가 더 적기 때문이다.)

### Reference
- https://velog.io/@zidru/%EC%BB%B4%ED%8C%8C%EC%9D%BC%EA%B3%BC-%EB%9F%B0%ED%83%80%EC%9E%84-%EA%B7%B8%EB%A6%AC%EA%B3%A0-%EC%9D%B8%ED%84%B0%ED%94%84%EB%A6%AC%ED%84%B0
- https://blog.ukjae.io/posts/inspecting-list-comprehension/
- https://docs.python.org/ko/3.11/library/dis.html#opcode-CALL
- https://github.com/python/cpython/blob/3.11/Python/ceval.c#L4202